自定义神经网络层和块的学习

In [30]:
# import the packages
import torch
import numpy
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn

%matplotlib inline

In [31]:
class LayerLinear(nn.Module):
    """
    自定义线性层
    """
    def __init__(self, inFeature, outFeature) -> None:
        super().__init__()

        self.W = nn.Parameter(torch.rand(inFeature, outFeature))
        self.B = nn.Parameter(torch.rand(outFeature, ))

    def forward(self, inputData):
        output = inputData @ self.W + self.B
        return F.relu(output)

In [32]:
testLayer = LayerLinear(10, 2)
testLayer(torch.rand(2, 10))

tensor([[1.8868, 1.9151],
        [1.8766, 2.0984]], grad_fn=<ReluBackward0>)

In [33]:
class LinearBlock(nn.Module):
    """
    自定义模块 (若干层的叠加)
    """
    def __init__(self, inFeature, outFeature) -> None:
        super().__init__()

        self.inputLayer = nn.Linear(inFeature, 128)
        self.outputLayer = nn.Linear(128, outFeature)
    
    def forward(self, inputData):
        X = F.relu(self.inputLayer(inputData))
        X = F.sigmoid(self.outputLayer(X))

        return X

In [34]:
testBlock = LinearBlock(10, 2)
testBlock(torch.rand(2, 10))

tensor([[0.4921, 0.4919],
        [0.4855, 0.5055]], grad_fn=<SigmoidBackward0>)

In [35]:
class FixedMLP(nn.Module):
    """
    自定义具有固定参数层 和 共享参数层 的 MLP
    
    固定参数实现:
        1. 在前向传播的时候取消不计算梯度 with torch.no_grad()
        2. 在设置其可训练参数的时候使 requires_grad = False
    """

    def __init__(self, inFeature, outFeature):
        super().__init__()

        self.inputLayer = nn.Linear(inFeature, 16)
        # 将 其 权重 全部赋值为 0.1
        self.inputLayer.weight.data.fill_(0.1)
        
        self.hidden = nn.Linear(16, 64)
        self.transfor = nn.Linear(64, 16)
        self.outputLayer = nn.Linear(64, outFeature)

    def forward(self, inputData):
        # 不加入梯度计算，就不会参与反向传播更新参数
        with torch.no_grad():
            X = F.leaky_relu(self.inputLayer(inputData))
            
        X = F.leaky_relu(self.hidden(X))
        X = F.leaky_relu(self.transfor(X))

        # 使用同一层. 说明共享参数
        X = F.leaky_relu(self.hidden(X))

        return self.outputLayer(X)

In [39]:
fMLP = FixedMLP(10, 2)
prediction = fMLP(torch.rand(2, 10))

loss = torch.sum(torch.abs(prediction - torch.ones(2, 2)))
loss.backward()
optimizer = torch.optim.SGD(fMLP.parameters(), lr = 0.9)
optimizer.step()
print(fMLP.inputLayer.weight)

tensor(3.7196, grad_fn=<SumBackward0>)
Parameter containing:
tensor([[0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000,